In [7]:
#Import from the Keras library
from keras import models
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,  MaxPooling2D
from keras import optimizers 
from keras import utils
from keras.models import load_model

#Other import statements 
import h5py
import random
import numpy as np
import pandas as pd
import pathlib
import cv2
import pymysql

In [1]:
#The values by which to resize images
nrows = 150
ncols = 150 
channels = 3

In [12]:
def connect(): 
    db_host ='nicu-2019-03-05.c2lckhwrw1as.us-east-1.rds.amazonaws.com'
    db_port = 3306
    db_name = 'nicu'
    db_username = 'jonlee'
    db_password = 'nicu_jon'
    
    conn = pymysql.connect(db_host, user=db_username, port=db_port, passwd=db_password, db=db_name)
    return conn

In [50]:
def import_data(conn=connect()): 
    '''
    '''
    curs = conn.cursor()
    
    recording_ids = {}
    xy = {}
    
    primary_query = "SELECT recording_id, id FROM Video_Raw WHERE recording_id>1 limit 10"
    
    try: 
        curs.execute(primary_query)
        for x in curs.fetchall(): 
            rec_id = x[0]
            raw_id = x[1]
            if rec_id in recording_ids:
                recording_ids.get(rec_id).append(raw_id)
            else: 
                recording_ids.update({rec_id:[raw_id]})
    except Exception as e:
        print("No ", e)
        
        
    image_query = """SELECT r.id, g.RGB_Optical_Flow, r.isCSGM 
                     FROM Video_Raw AS r 
                     JOIN Video_Generated AS g 
                     ON r.id as g.raw_id
                     WHERE r.recording_id>1 limit 10"""
    try:
        curs.execute(image_query) #(list(recording_ids.keys())))
        for x in curs.fetchall():
            raw_id = x[0]
            image = x[1] 
            xy.update({raw_id:[image,x[2]]})
            
    except Exception as e:
        print("No 1 ", e)
        
    label_query = "SELECT id, isCSGM FROM Video_Raw WHERE recording_id>1 limit 10"
    
    try:
        curs.execute(label_query)
        for x in curs.fetchall():
            xy.get(x[0]).append(x[1])
    
    except Exception as e:
        print("No 2 ", e)
    
    
    return recording_ids, xy
    


In [51]:
print(import_data())

({2: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]}, {12: [None, 0], 13: [b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x02\x00\x00\x00\xba\xb3K\xb3\x00\x00 \x00IDATx\x01\xec\xc1m\x8feWb\x1e\xd6\xf5\xecsoUw\x0f\xc9\xd1\xc8r\xfc\x92W \x81?\xe4\xfdK\x02#\x08\x90\xff\xff\x0b\x02$@\x1c\x18\t\x02A\xb2-i\x86\xdd\xac\xba\xe7\xec\'\xf7\xb2\x8al\x92\xc3\x19q4\xd3\xb4\xab{\xaf\x95\x7f\xfe\x8f\\]v\x8f\x17\x8f\xbb\xc7\xdd1\xfd\x0e!\xd4\xb3\x10\xea\xbd\xe9&\x9e\xd5M\xa8\xef\x88\x1f\xd1x\xb2\xf5\xf4\xc6g\x9f\xf9\xfcd\xbbxw\xf1x\xd8\xcb4\xa79\xf5\xd1\xc3\xf6Y\xb7\xd8\x1f\x1d\xbby\xf8\xbdF\x88\xe2\xe4\xfe\xe4\x8e9\x1d\x87K\xa5\x8e\x9an\x82\x8ef\x98\x93\xe9I\xee\x8d{6\x19\x84H\xd8\xb87?w\xfc\x85\xed+\xfe\xda\xfcW^\xef\xbfx\xe7]\xd5\xf7\xd4\x073\xe2[\x89\xc48i\xcc\xa9\x07\xd3\xac\xef\x8b\xf3k\xe3\x0b\x0f6\xef\xa6\xb7u\xf1}\xd9\x9c\xef\x9d\xee\xcc\xdd\xe5+\xc7\xee\xf7{u\xef|2\x86Db\xc4ixu\xf2\xfa\xde/\xee\xfc\xea3\xff\xcf\xbf\xf5\xaf\xff\xca\xdbG\xa6?L\xbcW\xdf\x15\xea\x07^\xc9\xd0

In [52]:
def create_array(raw_ids, xy):    
    image_list=[]
    csgm_list=[]
    
    random.shuffle(raw_ids)
    
    for i in raw_ids:
        if xy.get(i)[0]!=None:
            image_list.append(xy.get(i)[0])
            csgm_list.append(xy.get(i)[1])
    x = np.array(image_list)
    y = np.array(csgm_list)
    return x, y
        

In [6]:
def create_model():    
    model = models.Sequential() 

    #The model will learn 32 filters in this layer
    model.add(Conv2D(32, (3,3), activation = 'relu', input_shape=(150,150,3))) # this applies 32 convolution filters of size 3x3 each
    model.add(Conv2D(32, (3,3), activation='relu'))

    #Max Pooling takes a 2x2 grid and takes the highest value of the grid and resizes based on that max value
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    #Softmax activation is the traditional form of activation for mutliclass classification
    model.add(Dense(1, activation='softmax')) 
    
    model.compile(loss='categorical_hinge', 
              optimizer=optimizers.SGD(lr=1e-4), #SGD = Stochastic Gradient Descent (Can be changed)
              metrics=['acc']) 

    return model


In [7]:
def confusion_matrix(size, exp_values, predicted_values, labels):
    """
    This creates a confusion matrix with the predicted accuracy of the model.
    The ouput is a pandas dataframe with the labels and columns of the different labels inputted. 
    """
    predicted_values = convert_predictions(predicted_values)
    
    #Creates a DataFrame of zeros
    matrix = pd.DataFrame(np.zeros((size,size)) , labels, labels)
    
    #Caculates whether the score was right or wrong 
    for i in range(len(exp_values)):
        for j in range(size):
            if exp_values[i][j] == predicted_values[i][j] and predicted_values[i][j] == 1:
                matrix.iloc[[find_one(predicted_values[i])],[find_one(exp_values[i])]] += 1
            elif exp_values[i][j] == 0 and predicted_values[i][j] == 1:
                matrix.iloc[[find_one(predicted_values[i])],[find_one(exp_values[i])]] += 1
   
    #Calculate diagonal sum and the accuracy of the model
    diagonal_sum = 0
    for i in range(size):
        diagonal_sum += matrix.iloc[i][i]
    
    score = diagonal_sum/len(exp_values)
    
  
    return  matrix, score
    
    
def find_one(l):
    """
    Finds the first occurence of one in a list and returns it.
    """
    for i in range(len(l)): 
        if l[i] == 1: return i
    return 0
            
def convert_predictions(predictions): 
    """
    Converts predictions outputted by a keras model into a list with 1 represented the predicted output and zero 
    in other classes. 
    """
    for p in predictions: 
        max_val = max(p)
        for n in range(len(p)): 
            if p[n] == max_val:
                p[n] = 1
            else:
                p[n] = 0
    return predictions

In [8]:
def runTest():
    matrices = {}
    scores = {}
    model_scores = {}
    
    recording_ids_dict, xy = import_data()
    
    for i in recording_ids_dict:
        train_ids= []
        test_ids = []
        
        for j in recording_ids_dict:
            if j == i:
                test_ids = recording_ids_dict[j]
            else: 
                train_ids = train_ids.append(recording_ids_dict[j])
        
        x_train, y_train = create_array(train_ids, xy)
        x_test, y_test = create_array(test_ids, xy)
        
        model = create_model()
        
        #Fit the model
        model.fit(x_train, y_train, batch_size = 32, epochs = 5)
        
        #Create predictions and evaluate to find loss and accuaracy
        predict = model.predict(x_test)
        model_score = model.evaluate(x_test, y_test)
        
        matrix,score = confusion_matrix(len(y_test[0]), y_test, predict, labels)
        
        matrices.update({i : matrix})
        print(matrices)
        scores.update({i: score})
        print(scores)
        model_scores.update({i:model_score})
        
    return matrices, scores, model_scores 

In [10]:
labels = ["LeftArm", "RightArm", "LeftLeg","RightLeg"]
names = ["/Isaiah", "/Kaylee", "/Patterson", "/Ryan"]
matrices, scores, model_scores = runTest(names,labels)

with open('LargeHingeResults.txt', 'w') as f:
    for key in matrices:
        f.write("%s\n" % key)
        f.write("%s\n" % matrices[key])
        f.write("%s\n" % scores[key])
        f.write("%s\n" % model_scores[key])

['/Kaylee', '/Patterson', '/Ryan'] are the training names
/Isaiah is being tested on.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
2039/2039 [==============================] - 209s 102ms/step - loss: 0.9849 - acc: 0.4836
Epoch 2/5
2039/2039 [==============================] - 745s 365ms/step - loss: 0.4436 - acc: 0.7989
Epoch 3/5
2039/2039 [==============================] - 1188s 583ms/step - loss: 0.2830 - acc: 0.8872
Epoch 4/5
1504/2039 [=====================>........] - ETA: 49s - loss: 0.2363 - acc: 0.9122

KeyboardInterrupt: 